# IPython tips

Here are some functionalities and tips I tend to forget a lot. From https://wesmckinney.com/book/python-basics

## Introspection

Help with `?`

In [2]:
list?

Pattern match with `?` and `*`

In [3]:
list.c*?

## Type checking

`isinstance` can take tuple of types!

In [4]:
a = 1; b = 1.5
isinstance(a, (int, float))

True

## Duck typing

Checking whther an object has certain method. "If it walks like a duck and quaks like a duck, it is a duck."

In [5]:
def isiterable(obj):
    try:
         iter(obj)
         return True
    except TypeError: # not iterable
         return False

In [6]:
isiterable(list)

False

In [7]:
isiterable([1,2])

True

## Equality check

In [8]:
lst1 = [1,2]
lst2 = lst1
lst1 == lst2

True

In [9]:
lst1 is lst2

True

`lst1` and `lst2` are the same.

A new list object makes it different.

In [10]:
lst3 = list(lst1)
lst1 == lst3

True

In [11]:
lst1 is lst3

False

They are the same with `==` (equality operator) because they have same values, but different with `is` (identity operator) due to different memory locations.

`is` can be used to test whether something is `None` or not because there can only be one `None` in python.

In [12]:
None is None

True

## String template and format

In [13]:
template = "{0:.2f} {1:s} are worth US${2:d})"
template.format(40.333, "hihi", 3)

'40.33 hihi are worth US$3)'

- `{0:.2f}`: More on https://docs.python.org/3/library/string.html

## Popping dictionary

In [14]:
d1 = {'hi':'hola', 'bye':'adios'}
greeting = d1.pop('hi')
greeting

'hola'

## Files and operating syste

From https://wesmckinney.com/book/python-builtin#files_os

## Metaclasses

In new python, type of an object is class. Reading from [python-metaclasses](https://realpython.com/python-metaclasses/).

In [15]:
class Foo: pass
obj = Foo()
obj.__class__ is type(obj)

True

In [16]:
[].__class__ is type([])

True

All the built-in classes are `type`.

In [17]:
for t in int, float, dict, list, tuple: print(type(t))

<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>
<class 'type'>


In [18]:
type(type)

type

`type` is a metaclass, and classes are "instances".

### Creating classes dynamically

`type` is used to create classes as well with arguments.

`type(name, bases, dict, **kwds) -> a new type`

#### Example 1

The simplest class.

In [19]:
Foo = type('Foo', (), {})

x = Foo()
x

In [20]:
class Foo:
    pass

x = Foo()
x

#### Example 2

A class with a base and an attribute.

In [21]:
Bar = type('Bar', (Foo,), dict(attr=100))

x = Bar()
x.attr

100

In [22]:
x.__class__

__main__.Bar

In [23]:
x.__class__.__bases__

(__main__.Foo,)

In [24]:
class Bar(Foo):
    attr = 100


x = Bar()
x.attr

100

In [25]:
x.__class__

__main__.Bar

In [26]:
x.__class__.__bases__

(__main__.Foo,)

#### Example 3

A class with no base, but two attributes.

In [27]:
Foo = type(
    'Foo',
    (),
    {
        'attr': 100,
        'attr_val': lambda x : x.attr
    }
)

x = Foo()
x.attr

100

In [28]:
x.attr_val()

100

In [29]:
class Foo:
    attr = 100
    def attr_val(self):
        return self.attr


x = Foo()
x.attr

100

In [30]:
x.attr_val()

100

#### Example 4

Attributes can be defined externally.

In [31]:
def f(obj):
    print('attr =', obj.attr)

In [32]:
Foo = type(
    'Foo',
    (),
    {'attr': 100, 'attr_val': f}
)

x = Foo()
x.attr

100

In [33]:
x.attr_val()

attr = 100


In [34]:
class Foo:
    attr = 100
    attr_val = f


x = Foo()
x.attr

100

In [35]:
x.attr_val()

attr = 100


### Custom metaclasses

In [36]:
class Foo: pass

f = Foo()

When `Foo` creates an instance, it calls `__new__()` and `__init__()`.

In [37]:
def new(cls):
    print(cls)
    x = object.__new__(cls)
    x.attr = 100
    return x

Foo.__new__ = new

f = Foo()
f.attr

<class '__main__.Foo'>


100

In [38]:
g = Foo()
g.attr

<class '__main__.Foo'>


100

In [39]:
object.__new__

<function object.__new__(*args, **kwargs)>

In [74]:
class MyClass(Bar):
    def __new__(cls, *args, **kwargs):
        print("Creating instance...")
        instance = super().__new__(cls)
        return instance

    def __init__(self, value):
        print("Initializing instance...")
        self.value = value
    
obj = MyClass(10)

Creating instance...
Initializing instance...


Reassigning `type.__new__` is not allowed.

In [40]:
def new(cls):
    x = type.__new__(cls)
    x.attr = 100
    return x

# Error
# type.__new__ = new

To override `__new__`, we have to create a meta class. To create a metaclass, we derive from `type`. Meta class is a class factories. 

In [80]:
class Meta(type):
    def __new__(cls, name, bases, dct):
        print(f'Creating {name} class with parent {cls}')
        x = super().__new__(cls, name, bases, dct)
        x.attr = 100
        return x

We can specify `metaclass` keyword on function definitions to use this specific metaclass, rather than default `type`.

In [86]:
class Foo(metaclass=Meta): pass

Foo.attr

Creating Foo class with parent <class '__main__.Meta'>


100

In [87]:
class Foo:
    def __init__(self):
        self.attr = 100

In [88]:
x = Foo()
x.attr

100

In [89]:
x

In [90]:
y = Foo()
y.attr

100

In [91]:
z = Foo()
z.attr

100

In [92]:
class Meta(type):
    def __init__(cls, name, bases, dct):
        cls.attr = 100

In [93]:
class X(metaclass=Meta): pass
X.attr

100

In [50]:
class Y(metaclass=Meta): pass
Y.attr

100

In [51]:
class Z(metaclass=Meta): pass
Z.attr

100

Simple inheritance also does the same thing:

In [94]:
class Base: attr = 100

In [95]:
class X(Base): pass

X.attr

100

Or a class decorator:

In [96]:
def decorator(cls):
    class NewClass(cls):
        attr = 100
    return NewClass

In [98]:
@decorator
class X: pass

X.attr

100